# Model Encodeur EfficientNet 

**Objectif:** le but de ce notebook est d'utilisé l'encodeur EfficientNet et de comprendre les différentes partie de cette encodeur.
L'utilité de cet encodeur est le fait qu'il soit plus efficace et plus rapide car il utilie moins de paramètres.

![title](../img/compare_effnet.png)

### Root Variables 

In [1]:
import os 

In [2]:
root = '/home/ign.fr/ttea/Code_IGN/AerialImageDataset'
train_dir = os.path.join(root,'train/images')
gt_dir = os.path.join(root,'train/gt')
test_dir = os.path.join(root,'test/images')

In [3]:
import sys 

In [4]:
sys.path.insert(0, '/home/ign.fr/ttea/stage_segmentation_2021/Code')

In [5]:
from dataloader.dataloader import InriaDataset
from model.model import UNet
from train import train_segmentation, eval_segmentation, train_full_segmentation
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn
from efficientnet_pytorch import EfficientNet
from efficientnet_pytorch.utils import url_map, url_map_advprop, get_model_params

### Import Libraries 

In [6]:
import pandas as pd 

import torch
import torch.nn.functional as F
import torch.nn as nn

In [7]:
var= pd.read_json('variables.json')

## Inria Dataset

In [8]:
tile_size = (512,512)
train_dataset = InriaDataset(var['variables']['root'],tile_size,'train',None,False,1)
val_dataset = InriaDataset(var['variables']['root'],tile_size,'validation',None,False,1)

## EfficientNet Model

On va essayer différentes méthodes pour avoir des modèles plus ou moins efficace en faisant varier certains paramètres.

Compound Scaling (Mise à l'échelle par composition) : méthode utilisée pour pouvoir scale la largeur, profondeur & résolution ensemble. 

- **depth :** $d = \alpha^{\phi}$  
- **width :** $w = \beta^{\phi}$
- **resolution :** $r = \gamma^{\phi}$

Ou : $\alpha * \beta^2 * \gamma^2 \approx = 2$

$\alpha \geqslant 1, \beta \geqslant 1, \gamma \geqslant 1$

$ \alpha, \beta, \gamma$ sont des multiplicateurs d'échelle (scaling multiplier) pour la profondeur, largeur & résolution.  

![title](../img/compoundscaling.png)

![title](../img/performancescaling.png)

![title](../img/efficientnet.png)

Sur cette architecture, on peut voir qu'on utilise 7 inverted residual blocks mais chaque block à un setting différent. 

Ils utilisent aussi le squeeze & excitation block avec la fonction d'activation swish. 

### Swish Activation 

![title](../img/swish.png)

La fonction d'activation ReLU fonctionne plutôt bien mais il reste quelques problèmes au niveau des valeurs négatives et donc leurs dérivées sont toutes égales à 0 lors qu'on utilise des valeurs négatives. 

Pour palier ce problème, nous allons utiliser une nouvelle fonction d'activation "Swish".

Swish est une multiplication d'une fonction linéaire et de la sigmoid :
- $Swish(x) = x * sigmoid(x)$

### Inverted Residual Block 

Le principe d'un bloc résiduel est d'utiliser des convolutions en profondeur puis des faire des convolutions point par point. Cette approche permet de décroître le nombre de paramètres à entraîner. 

Dans un bloc résiduel original, les skip connections sont utilisées pour connecter une couche large (une couche avec un grand nombre de canaux) et ensuite de prendre quelques canaux à l'intérieur d un block (narrow layers) 

![title](../img/residualblock.png)

Le block résidual inversé fait le contraire, c'est-à-dire que les skips connections connectent les couches avec très peu de canaux (narrow layers) pendant que les wider layers sont entre les skips connections 

![title](../img/invertedblock.png)

### Squeeze & Excitation Block 

Le bloc de compression et d'excitation (SE) est une méthode pour donner un poids à chaque canal au lieu de les traiter tous de manière égale.

SE Block donne en sortie la forme $(1*1*canaux)$ qui spécifie le poids pour chaque canal. L'avantage est que le réseau de neurones peut apprendre ce poids par lui-même comme pour les autres paramètres.

![title](../img/SEblock.png)

### MBConv Block  

MBConv block prends 2 entrées: 
- La donnée 
- Block en arguments

La donnée est récupérée en sortie de la dernière couche. 

Le block argument est une collection d'attribut qui peut être utilisée à l'intérieur du MBConv block comme par exemple le nombre d'entrée ou de sortie pour les filtres, taux de dépenses, rapport de compression etc...


Dans notre cas, EfficientNet utilise 7 MBConv blocks avec quelques spécifications pour chaque block  :

- kernel size pour les convolutions sont 3x3
- nombre de répétition qui détermine combien de fois un block en particulier à besoin d'être répétée (doit être plus grand que 0)
- Nombre d'entrée et de sortie pour les filtres 
- Taux de dépenses (expand ratio) filtre d'entrée pour le taux de dépenses 
- Id skip : détermine si on utilise le skip connection ou non 
- se ratio : détermine le rapport de compression pour le block SE 

### Les Différentes Phases 

- Expansion Phase 
- Depthwise Convolution Phase
- Squeeze & Excitation Phase 
- Output Phase 

### Expansion Phase 

Nous allons élarger la couche et la rendre plus large (comme on l'a mentionné dans l'inverted residual block) afin d'augmenter le nombre de canaux. 

### Dephtwise convolution phase 

Après l'élargissement, nous allons utiliser des convolutions en profondeur. 

### Squeeze & Excitation phase 

Puis, nous allons extraire les principaux caractéristiques avec Global average pooling & squeeze numbers des canaux en utilisant le SE ratio. 

### Output phase 

Après avoir obtenu le block SE, on applique une opération de convolution qui nous donnera le filtre en sortie. 

### EfficientNet Model

![title](../img/efficientnet_archi.jpg)

**Documentation :**

- https://ai.googleblog.com/2019/05/efficientnet-improving-accuracy-and.html

- https://medium.com/analytics-avidhya/image-classification-with-efficientnet-better-performance-with-computational-efficiency-f480fdb00ac6

- https://towardsdatascience.com/mobilenetv2-inverted-residuals-and-linear-bottlenecks-8a4362f4ffd5

- https://amaarora.github.io/2020/08/13/efficientnet.html

### Implementation EfficientNet  

Eff-Unet Paper : https://openaccess.thecvf.com/content_CVPRW_2020/papers/w22/Baheti_Eff-UNet_A_Novel_Architecture_for_Semantic_Segmentation_in_Unstructured_Environment_CVPRW_2020_paper.pdf

EfficientUnet Github : https://github.com/zhoudaxia233/EfficientUnet-PyTorch/tree/master/efficientunet

https://github.com/lukemelas/EfficientNet-PyTorch


In [9]:
class Conv2dReLU(nn.Sequential):
    def __init__(
            self,
            in_channels,
            out_channels,
            kernel_size,
            padding=0,
            stride=1,
            use_batchnorm=True,
    ):

        if use_batchnorm == "inplace" and InPlaceABN is None:
            raise RuntimeError(
                "In order to use `use_batchnorm='inplace'` inplace_abn package must be installed. "
                + "To install see: https://github.com/mapillary/inplace_abn"
            )

        conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride=stride,
            padding=padding,
            bias=not (use_batchnorm),
        )
        relu = nn.ReLU(inplace=True)

        if use_batchnorm == "inplace":
            bn = InPlaceABN(out_channels, activation="leaky_relu", activation_param=0.0)
            relu = nn.Identity()

        elif use_batchnorm and use_batchnorm != "inplace":
            bn = nn.BatchNorm2d(out_channels)

        else:
            bn = nn.Identity()

        super(Conv2dReLU, self).__init__(conv, bn, relu)


class SCSEModule(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super().__init__()
        self.cSE = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, in_channels // reduction, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels // reduction, in_channels, 1),
            nn.Sigmoid(),
        )
        self.sSE = nn.Sequential(nn.Conv2d(in_channels, 1, 1), nn.Sigmoid())

    def forward(self, x):
        return x * self.cSE(x) + x * self.sSE(x)


class ArgMax(nn.Module):

    def __init__(self, dim=None):
        super().__init__()
        self.dim = dim

    def forward(self, x):
        return torch.argmax(x, dim=self.dim)


class Activation(nn.Module):

    def __init__(self, name, **params):

        super().__init__()

        if name is None or name == 'identity':
            self.activation = nn.Identity(**params)
        elif name == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif name == 'softmax2d':
            self.activation = nn.Softmax(dim=1, **params)
        elif name == 'softmax':
            self.activation = nn.Softmax(**params)
        elif name == 'logsoftmax':
            self.activation = nn.LogSoftmax(**params)
        elif name == 'tanh':
            self.activation = nn.Tanh()
        elif name == 'argmax':
            self.activation = ArgMax(**params)
        elif name == 'argmax2d':
            self.activation = ArgMax(dim=1, **params)
        elif callable(name):
            self.activation = name(**params)
        else:
            raise ValueError('Activation should be callable/sigmoid/softmax/logsoftmax/tanh/None; got {}'.format(name))

    def forward(self, x):
        return self.activation(x)


class Attention(nn.Module):

    def __init__(self, name, **params):
        super().__init__()

        if name is None:
            self.attention = nn.Identity(**params)
        elif name == 'scse':
            self.attention = SCSEModule(**params)
        else:
            raise ValueError("Attention {} is not implemented".format(name))

    def forward(self, x):
        return self.attention(x)


class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

In [10]:
class EncoderMixin:
    """Add encoder functionality such as:
        - output channels specification of feature tensors (produced by encoder)
        - patching first convolution for arbitrary input channels
    """

    @property
    def out_channels(self):
        """Return channels dimensions for each tensor of forward output of encoder"""
        return self._out_channels[: self._depth + 1]

    def set_in_channels(self, in_channels, pretrained=True):
        """Change first convolution channels"""
        if in_channels == 3:
            return

        self._in_channels = in_channels
        if self._out_channels[0] == 3:
            self._out_channels = tuple([in_channels] + list(self._out_channels)[1:])

        utils.patch_first_conv(model=self, new_in_channels=in_channels, pretrained=pretrained)

    def get_stages(self):
        """Method should be overridden in encoder"""
        raise NotImplementedError

    def make_dilated(self, output_stride):

        if output_stride == 16:
            stage_list=[5,]
            dilation_list=[2,]
            
        elif output_stride == 8:
            stage_list=[4, 5]
            dilation_list=[2, 4] 

        else:
            raise ValueError("Output stride should be 16 or 8, got {}.".format(output_stride))
        
        stages = self.get_stages()
        for stage_indx, dilation_rate in zip(stage_list, dilation_list):
            utils.replace_strides_with_dilation(
                module=stages[stage_indx],
                dilation_rate=dilation_rate,
            )

In [11]:
class EfficientNetEncoder(EfficientNet, EncoderMixin):
    def __init__(self, stage_idxs, out_channels, model_name, depth=5):

        blocks_args, global_params = get_model_params(model_name, override_params=None)
        super().__init__(blocks_args, global_params)
        #super().__init__()
        self._stage_idxs = stage_idxs
        self._out_channels = out_channels
        self._depth = depth
        self._in_channels = 3

        del self._fc

    def get_stages(self):
        return [
            nn.Identity(),
            nn.Sequential(self._conv_stem, self._bn0, self._swish),
            self._blocks[:self._stage_idxs[0]],
            self._blocks[self._stage_idxs[0]:self._stage_idxs[1]],
            self._blocks[self._stage_idxs[1]:self._stage_idxs[2]],
            self._blocks[self._stage_idxs[2]:],
        ]

    def forward(self, x):
        stages = self.get_stages()

        block_number = 0.
        drop_connect_rate = self._global_params.drop_connect_rate
        #drop_connect_rate  = 0.2
        features = []
        for i in range(self._depth + 1):

            # Identity and Sequential stages
            if i < 2:
                x = stages[i](x)

            # Block stages need drop_connect rate
            else:
                for module in stages[i]:
                    drop_connect = drop_connect_rate * block_number / len(self._blocks)
                    block_number += 1.
                    x = module(x, drop_connect)

            features.append(x)

        return features

    def load_state_dict(self, state_dict, **kwargs):
        state_dict.pop("_fc.bias", None)
        state_dict.pop("_fc.weight", None)
        super().load_state_dict(state_dict, **kwargs)


def _get_pretrained_settings(encoder):
    pretrained_settings = {
        "imagenet": {
            "mean": [0.485, 0.456, 0.406],
            "std": [0.229, 0.224, 0.225],
            "url": url_map[encoder],
            "input_space": "RGB",
            "input_range": [0, 1],
        },
        "advprop": {
            "mean": [0.5, 0.5, 0.5],
            "std": [0.5, 0.5, 0.5],
            "url": url_map_advprop[encoder],
            "input_space": "RGB",
            "input_range": [0, 1],
        }
    }
    return pretrained_settings

In [12]:
img, mask = train_dataset[42]
Encoder = EfficientNetEncoder((3,5,9,16),[3,32,24,40,112,320],"efficientnet-b0")
encoder = Encoder(img[None,:,:,:])

## Import EfficientNet Segmentation Model 

https://github.com/qubvel/segmentation_models

### EffificentNet 

In [13]:
efficientnet = smp.Unet(
    encoder_name="efficientnet-b0",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=2,                      # model output channels (number of classes in your dataset)
)

### Arguments & Hyperparamètres 

In [14]:
hparam = {
    'lr':0.0001,
    'n_epoch':5,
    'n_epoch_test':int(5),
    'n_class':1,
    'batch_size':8,
    'n_channel':3,
    'conv_width':[16,32,64,128,256,128,64,32,16],
}

In [15]:
tile_size = (512,512)

weights = [0.5, 1.0]
class_weights = torch.FloatTensor(weights).cuda()

args = {
    'nn_loss':nn.BCEWithLogitsLoss(reduction="mean"),
    #'nn_loss':nn.CrossEntropyLoss(weight = class_weights,reduction="mean"),
    #'nn_loss':BinaryDiceLoss,
    #'loss_name':'BinaryDiceLoss',
     'loss_name': 'BinaryCrossentropy',
    # 'loss_name': 'Crossentropy',
    'threshold':0.5,
    'cuda':1,
    'class_names':['None','Batiment'],
    'save_model':False,
    'save_model_name':"unet_test_8_1.pth",
    'train_dataset':InriaDataset(var['variables']['root'],tile_size,'train',None,False,1),
    'val_dataset':InriaDataset(var['variables']['root'],tile_size,'validation',None,False,1),
}

### Training EfficientNet

In [17]:
#trained_model, metric_train,metric_test = train_full(args, efficientnet,hparam['lr'],hparam['n_epoch'],
#                                    hparam['n_epoch_test'],hparam['batch_size'],hparam['n_class'],
#                                    hparam['n_channel'])